In [41]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("Data Folder/bigml_59c28831336c6604c800002a.csv", encoding='utf-8', index_col=0)


In [42]:
# Check column names (features)
print("Column names:")
print(df.columns.tolist())

# Basic info about the dataset
print("\nDataset info:")
print(df.info())

# Shape of the dataset (rows, columns)
print(f"\nDataset shape: {df.shape}")

# First few rows
print("\nFirst 5 rows:")
print(df.head())

Column names:
['account length', 'area code', 'phone number', 'international plan', 'voice mail plan', 'number vmail messages', 'total day minutes', 'total day calls', 'total day charge', 'total eve minutes', 'total eve calls', 'total eve charge', 'total night minutes', 'total night calls', 'total night charge', 'total intl minutes', 'total intl calls', 'total intl charge', 'customer service calls', 'churn']

Dataset info:
<class 'pandas.core.frame.DataFrame'>
Index: 3333 entries, KS to TN
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   account length          3333 non-null   int64  
 1   area code               3333 non-null   int64  
 2   phone number            3333 non-null   object 
 3   international plan      3333 non-null   object 
 4   voice mail plan         3333 non-null   object 
 5   number vmail messages   3333 non-null   int64  
 6   total day minutes       3333 non-null   flo

In [43]:
features = ['account length', 'total day minutes', 'total day calls', 'total day charge',
           'total eve minutes', 'total eve calls', 'total eve charge',
           'total night minutes', 'total night calls', 'total night charge',
           'total intl minutes', 'total intl calls', 'total intl charge',
           'customer service calls', 'number vmail messages']


#Creating a Model Data Set and dropping missing values

In [44]:
# Convert churn to binary (1 for True, 0 for False)
df['churn_binary'] = df['churn'].astype(int)

# Handle missing values
df.dropna(inplace=True)

print(f"Dataset shape after cleaning: {df.shape}")
print(f"\nChurn distribution:")
print(df['churn'].value_counts())
print(f"Churn rate: {df['churn_binary'].mean():.3f}")

# Display first few rows
print(f"\nFirst 5 rows:")
print(df.head())

Dataset shape after cleaning: (3333, 21)

Churn distribution:
False    2850
True      483
Name: churn, dtype: int64
Churn rate: 0.145

First 5 rows:
       account length  area code phone number international plan  \
state                                                              
KS                128        415     382-4657                 no   
OH                107        415     371-7191                 no   
NJ                137        415     358-1921                 no   
OH                 84        408     375-9999                yes   
OK                 75        415     330-6626                yes   

      voice mail plan  number vmail messages  total day minutes  \
state                                                             
KS                yes                     25              265.1   
OH                yes                     26              161.6   
NJ                 no                      0              243.4   
OH                 no                  

In [47]:
# CORRELATION ANALYSIS

print("\n" + "="*60)
print("FINDING BEST PREDICTOR FOR SIMPLE MODEL")
print("="*60)

# Calculate correlations with churn
correlations = df[features + ['churn_binary']].corr()['churn_binary'].sort_values(key=abs, ascending=False)

print("Feature correlations with churn (sorted by absolute value):")
for feature, corr in correlations.items():
    if feature != 'churn_binary':
        print(f"{feature:<25}: {corr:>8.4f}")

# Find the feature most correlated with churn
feature_corrs = correlations.drop('churn_binary')
best_feature = feature_corrs.abs().idxmax()
print(f"\nBest single predictor: {best_feature} (correlation: {feature_corrs[best_feature]:.4f})")



FINDING BEST PREDICTOR FOR SIMPLE MODEL
Feature correlations with churn (sorted by absolute value):
customer service calls   :   0.2087
total day minutes        :   0.2052
total day charge         :   0.2052
total eve minutes        :   0.0928
total eve charge         :   0.0928
number vmail messages    :  -0.0897
total intl charge        :   0.0683
total intl minutes       :   0.0682
total intl calls         :  -0.0528
total night charge       :   0.0355
total night minutes      :   0.0355
total day calls          :   0.0185
account length           :   0.0165
total eve calls          :   0.0092
total night calls        :   0.0061

Best single predictor: customer service calls (correlation: 0.2087)


In [46]:
# 1. SIMPLE LOGISTIC REGRESSION MODEL

print("\n" + "="*60)
print("1. BUILD SIMPLE LOGISTIC REGRESSION MODEL")
print("="*60)

print(f"Building simple logistic regression using '{best_feature}'")
print("The feature with highest correlation with churn will be used for simple logistic regression.")

# Prepare data for simple logistic regression
exog_simple = sm.add_constant(df[[best_feature]])
endog = df['churn_binary']

# Build simple logistic regression model
simple_model = sm.Logit(endog, exog_simple)
simple_model_results = simple_model.fit()

print(f"\nSimple Logistic Regression Results:")
print("="*50)
print(simple_model_results.summary())


1. BUILD SIMPLE LOGISTIC REGRESSION MODEL
Building simple logistic regression using 'customer service calls'
The feature with highest correlation with churn will be used for simple logistic regression.
Optimization terminated successfully.
         Current function value: 0.394125
         Iterations 6

Simple Logistic Regression Results:
                           Logit Regression Results                           
Dep. Variable:           churn_binary   No. Observations:                 3333
Model:                          Logit   Df Residuals:                     3331
Method:                           MLE   Df Model:                            1
Date:                Mon, 21 Jul 2025   Pseudo R-squ.:                 0.04751
Time:                        13:59:23   Log-Likelihood:                -1313.6
converged:                       True   LL-Null:                       -1379.1
Covariance Type:            nonrobust   LLR p-value:                 2.404e-30
                          

In [49]:
 #INTERPRET SIMPLE MODEL METRICS

print("\n" + "="*60)
print("2. INTERPRET SIMPLE LOGISTIC REGRESSION METRICS")
print("="*60)

# Extract key metrics
pseudo_r_squared = simple_model_results.prsquared
log_likelihood = simple_model_results.llf
aic = simple_model_results.aic
model_p_value = simple_model_results.llr_pvalue
model_is_significant = model_p_value < 0.05

print("Model Performance Metrics:")
print(f"Pseudo R-squared (McFadden): {pseudo_r_squared:.4f}")
print(f"Log-Likelihood: {log_likelihood:.2f}")
print(f"AIC: {aic:.2f}")
print(f"Model p-value (LLR test): {model_p_value:.2e}")
print(f"Model is statistically significant (α=0.05): {model_is_significant}")

# Calculate classification accuracy on training data
predictions_prob = simple_model_results.predict(exog_simple)
predictions_class = (predictions_prob > 0.5).astype(int)
accuracy = (predictions_class == df['churn_binary']).mean()
print(f"Training accuracy: {accuracy:.4f}")

# Store for comparison
r_squared_simple = pseudo_r_squared
model_significant_simple = model_is_significant



2. INTERPRET SIMPLE LOGISTIC REGRESSION METRICS
Model Performance Metrics:
Pseudo R-squared (McFadden): 0.0475
Log-Likelihood: -1313.62
AIC: 2631.24
Model p-value (LLR test): 2.40e-30
Model is statistically significant (α=0.05): True
Training accuracy: 0.8560


In [50]:
# 3. INTERPRET SIMPLE MODEL PARAMETERS

print("\n" + "="*60)
print("3. INTERPRET SIMPLE LOGISTIC REGRESSION PARAMETERS")
print("="*60)

# Extract coefficients
intercept = simple_model_results.params['const']
slope = simple_model_results.params[best_feature]

# Extract p-values
intercept_pvalue = simple_model_results.pvalues['const']
slope_pvalue = simple_model_results.pvalues[best_feature]

print("Model Parameters:")
print(f"Intercept (β₀): {intercept:.6f} (p-value: {intercept_pvalue:.4f})")
print(f"Slope (β₁): {slope:.6f} (p-value: {slope_pvalue:.4f})")

# Calculate odds ratio
odds_ratio = np.exp(slope)
print(f"\nOdds Ratio: {odds_ratio:.4f}")

print(f"\nModel equation: log(odds) = {intercept:.6f} + {slope:.6f} * {best_feature}")
print(f"Probability equation: P(churn=1) = 1 / (1 + exp(-({intercept:.6f} + {slope:.6f} * {best_feature})))")

print(f"\nInterpretation:")
print(f"- For every 1-unit increase in {best_feature}, the odds of churn multiply by {odds_ratio:.4f}")
if odds_ratio > 1:
    print(f"- Higher {best_feature} is associated with INCREASED churn probability")
else:
    print(f"- Higher {best_feature} is associated with DECREASED churn probability")

# Store parameters for later use
m = slope
b = intercept


3. INTERPRET SIMPLE LOGISTIC REGRESSION PARAMETERS
Model Parameters:
Intercept (β₀): -2.490165 (p-value: 0.0000)
Slope (β₁): 0.396169 (p-value: 0.0000)

Odds Ratio: 1.4861

Model equation: log(odds) = -2.490165 + 0.396169 * customer service calls
Probability equation: P(churn=1) = 1 / (1 + exp(-(-2.490165 + 0.396169 * customer service calls)))

Interpretation:
- For every 1-unit increase in customer service calls, the odds of churn multiply by 1.4861
- Higher customer service calls is associated with INCREASED churn probability
